In [163]:
import requests
import json
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

import numpy as np
import pandas as pd

In [5]:
sido_code ={
	"서울":1,
	"인천":2,
	"대전":3,
	"대구":4,
	"광주":5,
	"부산":6,
	"울산":7,
	"세종시":8,
	"경기":31,
	"강원":32,
	"충북":33,
	"충남":34,
	"경북":35,
	"경남":36,
	"전북":37,
	"전남":38,
	"제주":39
    }

In [6]:
sido_code = {v:k for k,v in sido_code.items()}

In [177]:

sido = list(range(1,9))+ list(range(31,40))
sort = ['MAIN_B','MAIN_A']
result = pd.DataFrame()

for code in tqdm(sido):
    for main in sort:
        offset = 1
        url = f'https://korean.visitkorea.or.kr/api/v1/curation/list?offset={offset}&page=1&device=PC&type={main}&regionCode={code}&sequence=0'
        res = requests.get(url)
        group = json.loads(res.text)

        offset = group['data']['total']

        print('now_working :', sido_code[code], offset)
        url = f'https://korean.visitkorea.or.kr/api/v1/curation/list?offset={offset}&page=1&device=PC&type={main}&regionCode={code}&sequence=0'
                
        res = requests.get(url)
        group = json.loads(res.text)

        
        print('total_rows_number :', len(result))
        bag = []
        for data in group['data']['items']:
            
            try:
                city = sido_code[code]
                title = data['content']['title']
                # print(title)
                overView = data['content']['databaseMaster']['overView']
                
                # likeCount = data['content']['likeCount']     # 삭제됨
                # shareCount = data['content']['shareCount']
                # commentCount = data['content']['commentCount']
                cotId = data['content']['cotId']
                addr1 = data['content']['databaseMaster']['addr1']
                mapx = data['content']['databaseMaster']['mapx']
                mapy = data['content']['databaseMaster']['mapy']


                url2 = 'https://korean.visitkorea.or.kr/call'
                data = {
                        'cmd': 'TOUR_CONTENT_BODY_DETAIL',
                        'cotId': cotId,
                        'locationx': 0.0,
                        'locationy': 0.0,
                        'stampId': '1589345b-b030-11ea-b8bd-020027310001'
                }
                res = requests.post(url2,data=data)
                group = json.loads(res.text)

                # 11/22 추가
                infoCenter = group['body']['detail']['infoCenter']
                parking = group['body']['detail']['parking']

                
                catchtitle = group['body']['detail'].get('catchtitle', np.nan)

                if main == 'MAIN_B':
                    useTime = group['body']['detail'].get('openTime', np.nan)
                else:
                    useTime = group['body']['detail'].get('useTime', np.nan)

                
                        
                
                treatMenu = group['body']['detail'].get('treatMenu', np.nan)
                homepage = group['body']['detail']['homepage']
                contentType = group['body']['detail']['contentType']
                conLike = group['body']['detail']['conLike']
                conRead = group['body']['detail']['conRead']
                conShare = group['body']['detail']['conShare']
                cid = group['body']['detail']['cid']
                imgPath = 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=' + group['body']['detail']['imgPath']
                ### 여기까지

                tagName = group['body']['detail']['tagName']
                tagName = ','.join([i for i in tagName.split('|') if not str.isdigit(i[0])])

                # # 전처리
                # infoCenter = re.sub(r'<.*?>|&lt;|&gt;|\n', '', infoCenter)
                # parking = re.sub(r'<.*?>|&lt;|&gt;|\n', '', parking)
                # overView = re.sub(r'<.*?>|&lt;|&gt;|\n', '', overView)
                # useTime = re.sub(r'<.*?>|&lt;|&gt;|\n', '', useTime)
                # match = re.match(r'<a href="(.+?)"', homepage)
                # if match:
                #     homepage = match.group(1)

                append_dict ={
                    'id' : cid,
                    'city' : city,
                    'cityCode': code,
                    'contentType':contentType,
                    'title': title,
                    'catchtitle': catchtitle,
                    'overView': overView,
                    'treatMenu': treatMenu,
                    'conLike': conLike,
                    'conRead': conRead,
                    'conShare': conShare,
                    'imgPath': imgPath,
                    # 'likeCount': likeCount,
                    # 'shareCount': shareCount,
                    # 'commentCount': commentCount,
                    'addr': addr1,
                    'info': infoCenter,
                    'parking': parking,
                    'useTime': useTime,
                    'tagName': tagName

                    }
                
                if (group['body']['subArticle']):
                    for i in group['body']['subArticle']:
                        append_list = []
                        content = i['contentBody']
                        # content = re.sub(r'<.*?>|&lt;|&gt;|\n', '', content)
                        text = i['displayTitle'] + ':' + content
                        append_list.append( text )
                append_dict['detail'] = ','.join(append_list)
                append_dict['lat'] = mapx
                append_dict['lon'] = mapy

                bag.append(append_dict)
                # print(append_dict)
            except Exception as e:
                print('this is error--------\n', e)
                continue

        result = pd.concat([result, pd.DataFrame(bag)], ignore_index=True )



    

  0%|          | 0/17 [00:00<?, ?it/s]

now_working : 서울 195
total_rows_number : 0
now_working : 서울 160
total_rows_number : 195


  6%|▌         | 1/17 [01:11<19:00, 71.27s/it]

now_working : 인천 196
total_rows_number : 355
now_working : 인천 156
total_rows_number : 551


 12%|█▏        | 2/17 [02:24<18:02, 72.18s/it]

...

 88%|████████▊ | 15/17 [32:19<05:12, 156.11s/it]

now_working : 전남 318
total_rows_number : 9462
now_working : 전남 388
total_rows_number : 9780


 94%|█████████▍| 16/17 [34:28<02:27, 147.99s/it]

now_working : 제주 188
total_rows_number : 10168
this is error--------
 'mapx'
now_working : 제주 162
total_rows_number : 10355


100%|██████████| 17/17 [35:47<00:00, 126.31s/it]


In [178]:
result

,id,city,cityCode,contentType,title,catchtitle,overView,treatMenu,conLike,conRead,conShare,imgPath,addr,info,parking,useTime,tagName,detail,lat,lon
0,2837964,서울,1,39,봉희설렁탕 본점,40년 전통의 은평구 설렁탕집,봉희설렁탕은 서울 은평구 신사동에 있다. 6호선 새절역 4번 출구 인근에 있다. 이...,"설렁탕, 양곰탕, 설곰탕, 얼큰설곰탕, 특설렁탕, 꼬리곰탕, 양무침, 황태구이, 도...",1,351,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 증산로 413 동현빌딩,02-302-9754,있음 (발렛파킹),06:00~22:00,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식",부대시설:무선인터넷,126.9138622709,37.5931861011
1,2845652,서울,1,39,벙구갈비 응암점,응암동 전통 수제 숙성 갈비 전문점,벙구갈비 응암점은 지하철 6호선 응암역 4번 출구 은명초등학교 부근에 있다. 매장 ...,"수제갈비 (350g), 수제생갈비 (300g), 수제그린포크 (250g), \n수제...",1,492,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 응암로 269,02-352-5092,있음,매일 11:00 ~ 23:00,"벙구갈비,벙구갈비응암점,은평구맛집,은평구맛집,음식,응암동맛집,응암역맛집",화장실:남녀 구분,126.9187839632,37.5960608545
2,2757826,서울,1,39,영동설렁탕,NaN,30여 년간 한자리에서 운영된 이 식당은 아직도 허름한 식당 외관을 고수하고 있다....,국밥,0,959,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 서초구 강남대로101안길 24,02-543-4716,있음(40대주차가능),00:00 - 24:00,"서초국밥,서초설렁탕,서초수육,서초영동설렁탕,영동설렁탕,음식",화장실:있음(남/녀 구분)\n,127.0174022351,37.5162107911
3,398352,서울,1,39,마복림원조할머니집떡볶이,NaN,"50년 이어온 전통의 맛, 떡볶이 대표 브랜드로 유명하다. 서울특별시 중구에 있는 ...",떡볶이 / 튀김,2,2238,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 다산로35길 5,02-2232-8930,가능,09:00-00:00,"떡볶이,마복림떡볶이,맛집,서울맛집추천,신당동맛집,음식","화장실:업소내, 남녀공용 화장실",127.0151150008,37.563221956
4,2721659,서울,1,39,태극당,서울에서 가장 맛있고 오래된 빵집,태극당은 서울에서 가장 오래된 빵집으로 1945년 광복의 때를 기다린 듯 우리 민족...,모나카 / 버터케익 / 야채 사라다 / 고방 카스테라 등,7,1296,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 동호로24길 7,02-2279-3152,,08:00~21:00,"데이트,베이커리,복고풍,신선한,옛날맛,옛날빵,옛날빵집,옛스러운분위기,음식,체인점",화장실:남/녀 화장실 구분,127.0051076826,37.5595805325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10512,126440,제주,39,12,안덕계곡,자연 풍광을 즐기는 계곡,안덕계곡은 서귀포 안덕면의 감산리 마을을 지나 바다로 유입되는 창고천 하류에 형성된...,NaN,2,1974,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 일주서로 1524,064-710-6653 (제주도청세계유산본부:상록수림담당)<br>\n064-732-...,주차 가능,상시개방,"경치좋은곳,계곡,관광지,생태관광지,안덕계곡,연인과함께,자연,자연좋은곳,제주권",입 장 료:무료,126.3529933785,33.2584404427
10513,759595,제주,39,14,제주도립미술관,제주다운 미술관,제주도립미술관은 아름답고 청정한 제주의 자연 속에 세워진 ‘예술의 빛과 숨결이 넘치...,NaN,2,1662,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 1100로 2894-78,064-710-4300,가능,09:00~18:00 (7월~9월 09:00~20:00)<br>\n※ 매표는 관람시...,"문화시설,문화예술,문화예술공간,미술관,복합문화공간,산책하기좋은곳,실내여행지,연인과함...","대관안내:[대관시설] 기획전시실(대), 기획전시실(소), 시민갤러리, 강당, 세미나...",126.4897521473,33.4525489265
10514,126443,제주,39,12,화순금모래해변,검은 모래밭이 펼쳐진 해변이 아름다운 곳,"화순금모래 해변은 99,174m²(약 3만 평)의 검은 모래밭이 펼쳐진 곳으로 주변...",NaN,3,2102,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 화순해안로,064-760-4372 (안덕면 사무소),있음,상시개방(해수욕장은 매년 상이),"가족여행,관광지,바다풍경,바다풍경여행,아이와함께,여름방학가볼만한곳,여름여행,여름휴가...",화장실:있음(남/녀구분),126.333514908,33.2407004292
10515,2414812,제주,39,12,신산공원,여유롭게 산책하며 휴식할 수 있는 공원,신산공원은 올림픽 성화의 국내도착을 기념하는 기념물이 세워져 있는 88 서울 올림픽...,NaN,1,1125,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 신산로 92-12,064-726-0885,있음,,"걷기길,관광지,나들이,당일치기여행,산책길,산책코스,산책하기,산책하기좋은곳,신산공원,...",화장실:있음(남/녀구분),126.533791569,33.505279902


In [179]:
result.to_csv('./travel_spot.csv')